Register workspace

In [104]:
from azureml.core import Workspace 

In [105]:
ws= Workspace.from_config(path="./file-path/ws_config.json")

Choose existing experiment

In [106]:
from azureml.core.experiment import Experiment

In [107]:
Experiment.list(ws)

[Experiment(Name: mslearn-bike-rental,
 Workspace: Experiments),
 Experiment(Name: BrainStar,
 Workspace: Experiments),
 Experiment(Name: Hello_World1,
 Workspace: Experiments)]

In [108]:
experiment = Experiment(workspace=ws, name='BrainStar')

Get datastore communication

In [109]:
from azureml.core import Datastore
def_data_store = ws.get_default_datastore()

In [110]:
def_blob_store = Datastore(ws, "workspaceblobstore")

In [111]:
from azureml.core import Dataset
dataset = Dataset.get_by_name(ws, name='Absence data')
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/10-01-2020_021337_UTC/absence_record.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "a7741100-71c6-482a-929d-09c835916599",
    "name": "Absence data",
    "version": 1,
    "workspace": "Workspace.create(name='Experiments', subscription_id='516f235b-0a3a-42a9-b501-b6952a5cdfc5', resource_group='Experiments')"
  }
}

Set compute target

In [112]:
compute_target = ws.compute_targets["BrainStar1"]

In [113]:
compute_target

AmlCompute(workspace=Workspace.create(name='Experiments', subscription_id='516f235b-0a3a-42a9-b501-b6952a5cdfc5', resource_group='Experiments'), name=BrainStar1, id=/subscriptions/516f235b-0a3a-42a9-b501-b6952a5cdfc5/resourceGroups/Experiments/providers/Microsoft.MachineLearningServices/workspaces/Experiments/computes/BrainStar1, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags=None)

In [114]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 
aml_run_config = RunConfiguration()
# `compute_target` as defined in "Azure Machine Learning compute" section above
aml_run_config.target = compute_target

In [115]:
USE_CURATED_ENV = False
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="BrainStar")
    aml_run_config.environment = curated_environment
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    
    # Add some packages relied on by data prep step
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
        conda_packages=['pandas','scikit-learn'], 
        pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]'], 
        pin_sdk_version=False)

In [116]:
from azureml.pipeline.core import PipelineData

output_data1 = PipelineData(
    "output_data1",
    datastore=def_blob_store,
    output_name="output_data1")

In [117]:
from azureml.pipeline.steps import PythonScriptStep
dataprep_source_dir = "./process"
entry_point = "process.py"
# `my_dataset` as defined above
ds_input = dataset.as_named_input('input1')

# `output_data1`, `compute_target`, `aml_run_config` as defined above
data_prep_step = PythonScriptStep(
    script_name=entry_point,
    source_directory=dataprep_source_dir,
    arguments=["--input", ds_input, "--output", output_data1],
    inputs=[ds_input],
    outputs=[output_data1],
    compute_target=compute_target,
    runconfig=aml_run_config,
    allow_reuse=True
)

In [118]:
steps=data_prep_step

In [119]:
from azureml.pipeline.core import Pipeline
pipeline1 = Pipeline(workspace=ws, steps=steps)

In [120]:
pipeline1.validate()

Step process.py is ready to be created [d95d964a]


[]

In [121]:
pipeline_run1 = Experiment(ws, 'Hello_World1').submit(pipeline1, regenerate_outputs=False)

Created step process.py [d95d964a][cb05a572-6bd2-434a-b2ce-59bf0701a918], (This step will run and generate new outputs)
Submitted PipelineRun ef31b884-8a9b-4fa2-94ee-1fd692acdfe3
Link to Azure Machine Learning studio: https://ml.azure.com/experiments/Hello_World1/runs/ef31b884-8a9b-4fa2-94ee-1fd692acdfe3?wsid=/subscriptions/516f235b-0a3a-42a9-b501-b6952a5cdfc5/resourcegroups/Experiments/workspaces/Experiments
